# Load packages and Install "aletheia-dnn" (if needed)

In [1]:
# !pip install aletheia-dnn

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Load data

In [3]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data = load_breast_cancer()
x = MinMaxScaler().fit_transform(data.data)
y = data.target
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=0)

Train PLNN

In [4]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras


def ReluNetClassifier(input_dim, hidden_layer_sizes, regl1=0, regl2=0, lr=0.001, random_state=0):

    np.random.seed(random_state) 
    tf.random.set_seed(random_state)

    hidden_layer_sizes = [input_dim] + hidden_layer_sizes
    model = keras.models.Sequential()

    if (regl1 > 0) & (regl2 == 0):
        regularizer = keras.regularizers.l1(regl1)
    elif (regl1 == 0) & (regl2 > 0):
        regularizer = keras.regularizers.l2(regl2)
    else:
        regularizer = keras.regularizers.l1_l2(l1=regl1, l2=regl2)
    
    for i in range(len(hidden_layer_sizes) - 1):
        model.add(keras.layers.Dense(
            input_dim=hidden_layer_sizes[i],
            units=hidden_layer_sizes[i + 1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='relu',
            kernel_regularizer=regularizer))

    model.add(keras.layers.Dense(
            units=1,
            input_dim=hidden_layer_sizes[-1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='sigmoid',
            kernel_regularizer=regularizer))

    # compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='binary_crossentropy')
    return model


def ReluNetRegressor(input_dim, hidden_layer_sizes, regl1=0, regl2=0, lr=0.001, random_state=0):

    np.random.seed(random_state) 
    tf.random.set_seed(random_state)

    hidden_layer_sizes = [input_dim] + hidden_layer_sizes
    model = keras.models.Sequential()

    if (regl1 > 0) & (regl2 == 0):
        regularizer = keras.regularizers.l1(regl1)
    elif (regl1 == 0) & (regl2 > 0):
        regularizer = keras.regularizers.l2(regl2)
    else:
        regularizer = keras.regularizers.l1_l2(l1=regl1, l2=regl2)

    for i in range(len(hidden_layer_sizes) - 1):
        model.add(keras.layers.Dense(
            input_dim=hidden_layer_sizes[i],
            units=hidden_layer_sizes[i + 1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='relu',
            kernel_regularizer=regularizer))

    model.add(keras.layers.Dense(
            units=1,
            input_dim=hidden_layer_sizes[-1],
            kernel_initializer='he_normal',
            bias_initializer='zeros',
            activation='linear',
            kernel_regularizer=regularizer))

    # compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                    loss='mean_squared_error')
    return model

In [5]:
mlp = ReluNetClassifier(train_x.shape[1], [40] * 4, regl1=0.01, lr=0.001, random_state=0)
mlp.fit(train_x, train_y, epochs=2000, batch_size=200, verbose=False)

Use OpenBox to interpret the fitted PLNN

In [6]:
from aletheia import *

coefs = [layer.kernel.numpy() for layer in mlp.layers]
intercepts = [layer.bias.numpy() for layer in mlp.layers]
clf = UnwrapperClassifier(coefs, intercepts)
clf.fit(x, y)
clf.summary()

,Count,Response Mean,Response Std,Local AUC,Global AUC
0,278.0,0.352518,0.478616,0.994501,0.992733
1,134.0,0.985075,0.121709,0.594697,0.007267
2,73.0,1.000000,0.000000,NaN,0.730716
3,48.0,1.000000,0.000000,NaN,0.007267
4,15.0,0.000000,0.000000,NaN,0.992733
5,8.0,0.000000,0.000000,NaN,0.992733
6,7.0,0.000000,0.000000,NaN,0.992733
7,4.0,1.000000,0.000000,NaN,0.007267
8,2.0,1.000000,0.000000,NaN,0.992733
